In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
# Set a shorter path for TensorFlow Datasets
import os
os.environ['TFDS_DATA_DIR'] = 'C:/tensorflow_datasets'

# Load the Plant Village dataset
(ds_train, ds_test), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Extraction completed...: 0 file [03:45, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [03:45<00:00, 225.74s/ url]


ExtractError: Error while extracting C:\tensorflow_datasets\downloads\plant_village\data.mend.com_publ-file_data_tywb_file_d565-c1rDQyRTmE0CqGGXmH53WlQp0NWefMfDW89aj1A0m5D_A to C:\tensorflow_datasets\downloads\extracted\ZIP.data.mend.com_publ-file_data_tywb_file_d565-c1rDQyRTmE0CqGGXmH53WlQp0NWefMfDW89aj1A0m5D_A: [WinError 206] The filename or extension is too long: 'C:\\tensorflow_datasets\\downloads\\extracted\\incomplete.3f7243c7c5a84a39b66ad27096f719c9_ZIP.data.mend.com_publ-file_data_tywb_file_d565-c1rDQyRTmE0CqGGXmH53WlQp0NWefMfDW89aj1A0m5D_A\\Plant_leave_diseases_dataset_without_augmentation\\Apple___Apple_scab'
On windows, path lengths greater than 260 characters may result in an error. See the doc to remove the limitation: https://docs.python.org/3/using/windows.html#removing-the-max-path-limitation

In [3]:
# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image, label

In [4]:
# Apply preprocessing
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

NameError: name 'ds_train' is not defined

In [5]:
# Build a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation='softmax')
])

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'ds_info' is not defined

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(ds_train, epochs=5, validation_data=ds_test)

In [ ]:
# Evaluate the model
loss, acc = model.evaluate(ds_test)
print(f"Test Loss: {loss:.2f}")
print(f"Test Accuracy: {acc:.2f}")
